# 데이터불러오기

In [1]:
import pandas as pd
import re
from collections import Counter
import requests
import bs4
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool, Manager

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel("./gu_list/서울_동작구.xlsx")
print(df.shape)
df

(2618, 7)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소
0,동작구,사당동,역전우동0410 & 빽다방 동작남성점,카페,197657.200839,442368.302789,https://place.map.kakao.com/25116156#comment
1,동작구,사당동,한강치킨(수제전문점),치킨,198276.099219,441987.393695,https://place.map.kakao.com/11515257#comment
2,동작구,상도1동,태능솔밭갈비,한식,195405.538042,444523.550310,https://place.map.kakao.com/11222981#comment
3,동작구,사당동,면장의 밥상,카페,197999.829570,441709.980023,https://place.map.kakao.com/1623686424#comment
4,동작구,사당동,클로이스 칠하우스,카페,197083.024969,442894.978971,https://place.map.kakao.com/505721130#comment
...,...,...,...,...,...,...,...
2613,동작구,상도1동,모녀당,카페,195446.090552,444613.146551,https://place.map.kakao.com/839496033#comment
2614,동작구,사당동,궁물푸드,한식,197870.258424,442312.433317,https://place.map.kakao.com/1572612979#comment
2615,동작구,상도동,양계장집아들 상도점,치킨,194940.235367,444041.600062,https://place.map.kakao.com/48395912#comment
2616,동작구,대방동,웰컴댕댕,카페,193947.700409,445210.645555,https://place.map.kakao.com/1591541759#comment


# 셀레니움과 양송이스프 

In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
from multiprocessing import Pool, Manager
options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
options.add_argument("--disable-extensions")
options.add_argument("disable-infobars") 
options.add_argument("window-size=1920x1080") 
options.add_argument("no-sandbox") 
options.add_argument("disable-gpu") 
options.add_argument("--lang=ko_KR") 
options.add_argument( 'user-agent = Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36')

columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=options)
cnt = 0
for i in df['리뷰주소']:
    try:
        driver.get(i)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')))
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"})
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    
        for rate, review in zip(rates, reviews):
            row = [names.text, rate.text[0], review.find(name="span").text]
            series = pd.Series(row, index=review_df.columns)
            review_df = review_df.append(series, ignore_index=True)

        for button_num in range(2, 6): 
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                row = [names.text, rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)
                cnt += 1
                if cnt % 100 == 0:
                    print(f'현재{cnt}개 진행완료')
    except:
        cnt += 1
        if cnt % 100 == 0:
            print(f'현재{cnt}개 진행완료')       
        pass
            
    
    
driver.close()

현재100개 진행완료
현재200개 진행완료
현재300개 진행완료
현재400개 진행완료
현재500개 진행완료
현재600개 진행완료
현재700개 진행완료
현재800개 진행완료
현재900개 진행완료
현재1000개 진행완료
현재1100개 진행완료
현재1200개 진행완료
현재1300개 진행완료
현재1400개 진행완료
현재1500개 진행완료
현재1600개 진행완료
현재1700개 진행완료
현재1800개 진행완료
현재1900개 진행완료
현재2000개 진행완료
현재2100개 진행완료
현재2200개 진행완료
현재2300개 진행완료
현재2400개 진행완료
현재2500개 진행완료
현재2600개 진행완료
현재2700개 진행완료
현재2800개 진행완료
현재2900개 진행완료
현재3000개 진행완료
현재3100개 진행완료
현재3200개 진행완료
현재3300개 진행완료
현재3400개 진행완료
현재3500개 진행완료
현재3600개 진행완료
현재3700개 진행완료
현재3800개 진행완료
현재3900개 진행완료
현재4000개 진행완료
현재4100개 진행완료
현재4200개 진행완료
현재4300개 진행완료
현재4400개 진행완료
현재4500개 진행완료
현재4600개 진행완료
현재4700개 진행완료
현재4800개 진행완료
현재4900개 진행완료
현재5000개 진행완료
현재5100개 진행완료
현재5200개 진행완료
현재5300개 진행완료
현재5400개 진행완료
현재5500개 진행완료
현재5600개 진행완료
현재5700개 진행완료
현재5800개 진행완료
현재5900개 진행완료
현재6000개 진행완료
현재6100개 진행완료
현재6200개 진행완료
현재6300개 진행완료
현재6400개 진행완료
현재6500개 진행완료
현재6600개 진행완료
현재6700개 진행완료
현재6800개 진행완료
현재6900개 진행완료
현재7000개 진행완료
현재7100개 진행완료
현재7200개 진행완료
현재7300개 진행완료
현재7400개 진행완료
현재7500개 진행완료
현재7600개 진행완료
현재7700개 진행완료
현재7800개 

In [7]:
print('끝!')

끝!


In [8]:
review_df.shape

(13650, 3)

In [ ]:
# 강남구     8954 - 진행중
# 마포구     6106 - 진행중 
# 송파구     5446
# 영등포구    5024
# 서초구     4414
# 강서구     4376
# 관악구     3950
# 광진구     3562
# 동대문구    3394
# 강동구     3376
# 종로구     3291
# 구로구     3265
# 중랑구     3162
# 은평구     3132
# 성북구     3085
# 노원구     3013
# 중구      3012
# 성동구     3007
# 강북구     2905
# 서대문구   2626
# 동작구     2618
# 양천구     2613
# 용산구     2507
# 금천구     2433 - 진행중 
# 도봉구     2069(약 3시간)
# 구 별 가게 수예요,


In [5]:
review_df

,name,score,review
0,역전우동0410 동작남성점,1,포장주문 했는데 집에 와서 확인하니 상품이 누락되어있어 연락하니 와서 찾아가라고 그...
1,역전우동0410 동작남성점,1,1년전쯤인가 일하시는 이모님 한국분에서 중국분으로 바뀌셨는데 맛이 없어짐.. 전 차...
2,역전우동0410 동작남성점,1,역전우동 수십곳을 다녀봤는데 여기가 제일 맛없어요..하필 집근처지점이라 더 화나네요
3,역전우동0410 동작남성점,1,오자마자 가위를 좌우로 휘저으면서 자리 옮기라고 하는건 아니죠…못 앉을 곳 앉았나요...
4,역전우동0410 동작남성점,1,"맛없어요.,,. 배고파서 먹었다"
...,...,...,...
2071,흑수돈,5,무한리필에 빠삭빠삭한 튀김까지
2072,흑수돈,5,
2073,흑수돈,5,죽입니다
2074,수정싱싱회,5,대만족^^


In [10]:
review_df.to_csv('./서울_동작구.csv',encoding='utf-8-sig',index=False)

In [11]:
df_d = pd.read_csv('./서울_동작구.csv')

In [12]:
df_d

,name,score,review
0,역전우동0410 동작남성점,1,포장주문 했는데 집에 와서 확인하니 상품이 누락되어있어 연락하니 와서 찾아가라고 그...
1,역전우동0410 동작남성점,1,1년전쯤인가 일하시는 이모님 한국분에서 중국분으로 바뀌셨는데 맛이 없어짐.. 전 차...
2,역전우동0410 동작남성점,1,역전우동 수십곳을 다녀봤는데 여기가 제일 맛없어요..하필 집근처지점이라 더 화나네요
3,역전우동0410 동작남성점,1,오자마자 가위를 좌우로 휘저으면서 자리 옮기라고 하는건 아니죠…못 앉을 곳 앉았나요...
4,역전우동0410 동작남성점,1,"맛없어요.,,. 배고파서 먹었다"
...,...,...,...
13645,흥부왕족발부흥보쌈,4,존맛임~ 이집 항상 사람들 바글바글 다 이유가 있슴
13646,흥부왕족발부흥보쌈,1,아니 ㅋㅋㅋㅋ 족발 왜이래요??? 지방 90프로에 살 10프로??? 이곳은 추천하...
13647,흥부왕족발부흥보쌈,3,족발은 뻑뻑함. 맛없음. 근데 같이 나오는 국물이 예술임. 그것때문에 먹음. ...
13648,흥부왕족발부흥보쌈,1,여자분들은 가면 안될듯요. 이렇게 얇게 썬 족발은 살다살다 처음 봐요. 특대 대 중...
